In [ ]:
#!pip install nltk

In [82]:
import numpy as np
import pandas as pd
from SportsExperiencePlatform.data import connect_db, get_data
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import save_npz, load_npz

In [22]:
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /home/daniel/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
conn = connect_db()

if conn:
    users_df, events_df = get_data(conn)

In [4]:
users_df.shape, events_df.shape

((9, 13), (155, 17))

In [6]:
events_df.sample(10)

,id,title,description,price_per_person,capacity,address,latitude,longitude,is_external,url,offer_date,offer_time,created_at,updated_at,user_id,tag_id,external_image_url
65,171,Soccer (Wollishofen or Allmand),We play every Tuesday inside at the moment. Th...,5,NaN,no address provided,47.354008,8.520648,True,https://www.meetup.com/Outdoor-Soccer-spring-a...,2022-05-31,18:30:00,2022-05-24 19:25:59.417797,2022-05-24 19:25:59.417797,None,36,https://secure-content.meetupstatic.com/images...
64,154,2022 UEFA Champions League Final Home Viewing,Home Viewing Party for the 2022 UEFA Champions...,0,NaN,no address provided,52.571514,13.380952,True,https://www.meetup.com/berlin-pankow-european-...,2022-05-28,20:00:00,2022-05-24 19:25:59.321958,2022-05-24 19:25:59.321958,None,36,https://secure-content.meetupstatic.com/images...
125,216,Tennis Personal Coaching,no description,40,NaN,no address provided,0.000000,0.000000,True,https://www.meetup.com/Mainz-Tennis-Meetup/eve...,2022-05-29,11:00:00,2022-05-24 19:26:32.022300,2022-05-24 19:26:32.022300,None,39,https://secure-content.meetupstatic.com/images...
30,121,Monday Night Basketball at Hubelmatt,Throughout winter we meet on Mondays at the sp...,0,5.0,Zihlmattweg 2,47.035850,8.307271,True,https://www.meetup.com/Luzern-and-Central-Swit...,2022-05-30,20:00:00,2022-05-24 19:25:38.277613,2022-05-24 19:25:38.277613,None,32,https://secure-content.meetupstatic.com/images...
41,131,Climbing/Bouldering in Smichov,no description,0,5.0,no address provided,0.000000,0.000000,True,https://www.meetup.com/Getting-Active-in-Pragu...,2022-06-11,16:45:00,2022-05-24 19:25:43.940314,2022-05-24 19:25:43.940314,None,33,https://secure-content.meetupstatic.com/images...
20,110,Half-court Pick-up Basketball,Half-court pick-up basketball at Het Marnix. F...,4,1.0,Marnixplein 1,52.378002,4.878494,True,https://www.meetup.com/Pick-up-Basketball-Amst...,2022-06-08,18:00:00,2022-05-24 19:25:38.203449,2022-05-24 19:25:38.203449,None,32,https://secure-content.meetupstatic.com/images...
19,109,Half-court Pick-up Basketball,Half-court pick-up basketball at Het Marnix. F...,4,1.0,Marnixplein 1,52.378002,4.878494,True,https://www.meetup.com/Pick-up-Basketball-Amst...,2022-06-01,18:00:00,2022-05-24 19:25:38.198900,2022-05-24 19:25:38.198900,None,32,https://secure-content.meetupstatic.com/images...
116,205,Learning how to swim in Diemen ( private pool),"If you are living in The Netherlands, by now, ...",27,NaN,Kriekenoord 17,52.336560,4.975146,True,https://www.meetup.com/your-personal-swim-coac...,2022-06-02,19:15:00,2022-05-24 19:26:10.828434,2022-05-24 19:26:10.828434,None,38,https://secure-content.meetupstatic.com/images...
66,155,Real Madrid v Liverpool (Champions League Fina...,The next event is ready. It's a big one. The C...,0,NaN,Börsenbrücke 10,53.548550,9.992379,True,https://www.meetup.com/hamburg-watching-footba...,2022-05-28,20:00:00,2022-05-24 19:25:59.329197,2022-05-24 19:25:59.329197,None,36,https://secure-content.meetupstatic.com/images...
105,195,Swimming in the Limmat,https://secure.meetupstatic.com/photos/event/a...,0,2.0,Lettensteg 10,47.385654,8.534333,True,https://www.meetup.com/HealthCube/events/28605...,2022-06-10,12:30:00,2022-05-24 19:26:10.774350,2022-05-24 19:26:10.774350,None,38,https://secure-content.meetupstatic.com/images...


In [11]:
def remove_punctuation(text):
    for s in text:
        if s in string.punctuation:
            text = text.replace(s, '')

    return text

In [15]:
def lower_text(text):
    return text.lower()

In [17]:
def remove_numbers(text):
    return ''.join(word for word in text  if not word.isdigit())

In [20]:
def remove_stop_words(text, language='english'):
    stop_words = set(stopwords.words(language))
    word_tokens = word_tokenize(text)
    
    return ' '.join([w for w in word_tokens if not w in stop_words])

In [24]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()

    return ' '.join([lemmatizer.lemmatize(word) for word in text.split(' ')])

In [14]:
events_df['description_punct'] = events_df.description.apply(remove_punctuation)
events_df[['description', 'description_punct']].head()

,description,description_punct
0,Plutarch | Pleasure in the job puts perfection...,Plutarch Pleasure in the job puts perfection ...
1,"Pythagoras | It's not what happens to you, but...",Pythagoras Its not what happens to you but ho...
2,Pythagoras | Rhetoric is the art of ruling the...,Pythagoras Rhetoric is the art of ruling the ...
3,Posidonius | The virtue of justice consists in...,Posidonius The virtue of justice consists in ...
4,Parmenides | Only the educated are free.,Parmenides Only the educated are free


In [16]:
events_df['description_lower'] = events_df.description_punct.apply(lower_text)
events_df[['description', 'description_lower']].head()

,description,description_lower
0,Plutarch | Pleasure in the job puts perfection...,plutarch pleasure in the job puts perfection ...
1,"Pythagoras | It's not what happens to you, but...",pythagoras its not what happens to you but ho...
2,Pythagoras | Rhetoric is the art of ruling the...,pythagoras rhetoric is the art of ruling the ...
3,Posidonius | The virtue of justice consists in...,posidonius the virtue of justice consists in ...
4,Parmenides | Only the educated are free.,parmenides only the educated are free


In [18]:
events_df['description_numbers'] = events_df.description_lower.apply(remove_numbers)
events_df[['description', 'description_numbers']].head()

,description,description_numbers
0,Plutarch | Pleasure in the job puts perfection...,plutarch pleasure in the job puts perfection ...
1,"Pythagoras | It's not what happens to you, but...",pythagoras its not what happens to you but ho...
2,Pythagoras | Rhetoric is the art of ruling the...,pythagoras rhetoric is the art of ruling the ...
3,Posidonius | The virtue of justice consists in...,posidonius the virtue of justice consists in ...
4,Parmenides | Only the educated are free.,parmenides only the educated are free


In [21]:
events_df['description_stopwords'] = events_df.description_numbers.apply(remove_stop_words, language='english')
events_df[['description', 'description_stopwords']].head()

,description,description_stopwords
0,Plutarch | Pleasure in the job puts perfection...,plutarch pleasure job puts perfection work
1,"Pythagoras | It's not what happens to you, but...",pythagoras happens react matters
2,Pythagoras | Rhetoric is the art of ruling the...,pythagoras rhetoric art ruling minds men
3,Posidonius | The virtue of justice consists in...,posidonius virtue justice consists moderation ...
4,Parmenides | Only the educated are free.,parmenides educated free


In [25]:
events_df['description_lemmatize'] = events_df.description_stopwords.apply(lemmatize_text)
events_df[['description', 'description_lemmatize']].head()

,description,description_lemmatize
0,Plutarch | Pleasure in the job puts perfection...,plutarch pleasure job put perfection work
1,"Pythagoras | It's not what happens to you, but...",pythagoras happens react matter
2,Pythagoras | Rhetoric is the art of ruling the...,pythagoras rhetoric art ruling mind men
3,Posidonius | The virtue of justice consists in...,posidonius virtue justice consists moderation ...
4,Parmenides | Only the educated are free.,parmenides educated free


In [27]:
events_df[['description_lemmatize']].to_csv("test.csv")

In [28]:
tfidf = TfidfVectorizer(stop_words="english")

In [29]:
tfidf_matrix = tfidf.fit_transform(events_df["description_lemmatize"])
tfidf_matrix.shape

(155, 1690)

In [30]:
cosine_sim = tfidf_matrix.dot(tfidf_matrix.T)

In [91]:
cosine_sim

<155x155 sparse matrix of type '<class 'numpy.float64'>'
	with 11741 stored elements in Compressed Sparse Row format>

In [81]:
save_npz("model_cosine_sim", cosine_sim)

In [83]:
matrix_reloaded = load_npz("model_cosine_sim.npz")

In [84]:
matrix_reloaded.shape, cosine_sim.shape

((155, 155), (155, 155))

In [42]:
events_df[['description']].duplicated().sum()

79

In [43]:
title_to_index = pd.Series(events_df.index, index = events_df["title"]).drop_duplicates()

In [45]:
def content_recommender(title, cosine_sim = cosine_sim, df = events_df, title_to_index = title_to_index):
    idx = title_to_index[title]
    
    sim_scores = list(enumerate(cosine_sim[idx].todense().tolist()[0]))
    # list of set [(index, sim score), (index, sim score), ...]
    
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    # Sort the list of set by sim score, index with highest simlirarity
    # Will be at the beginning of the list
    
    sim_scores = sim_scores[1:11] # top 10 events (first one is our input events so we ignore it)
    
    sport_indices = [i[0] for i in sim_scores] # we grab the indices of those 10 events
    
    return df["title"].iloc[sport_indices]

In [88]:
events_df.title.unique()

array(['MLG Meadowlands', 'League All Stars', 'Serie A', 'ESL Cologne',
       'GFinity London', 'DFB Pokal', 'Super League', 'Worlds', 'Ligue 1',
       'Copa del Rey', 'Baseball @ Tempelhof',
       'Full court indoor Saturday Basketball in the centre of Rotterdam',
       "Casual Pick-up Women's Basketball",
       'Monday evening Basketball at Bergstr in Berlin Mitte 10115',
       'Basketball Sunday', 'Friday night basketball @ de Pijp',
       'Half-court Pick-up Basketball',
       'Sunday Basketball (INDOOR DOELENPLEIN) 🏀',
       'SPORTEGRATION Basketball', 'Basketball (outdoor)',
       'Monday Night Basketball at Hubelmatt',
       'Bouldering at Boulderwelt Frankfurt',
       'Bouldering at Boulderhaus DA',
       'Bouldering at StudioBloc Darmstadt',
       'Wednesday Evening Bouldering @BRIGHT SITE',
       'Soccer (Wollishofen or Allmand)',
       'Saturday Afternoon Bouldering @BRIGHT SITE',
       'Bouldering at Urban Apes St. Pauli (3G)',
       'Outside Bouldering in

In [90]:
#title = "Surf & Yoga Retreat Scheveningen"
#title = 'Bouldering at StudioBloc Darmstadt'
title = 'Beach Volleyball at Branik - OUTSIDE!'
content_recommender(title, cosine_sim = matrix_reloaded, df = events_df, title_to_index = title_to_index)
#content_recommender(title)

41                       Climbing/Bouldering in Smichov
42                       Climbing/Bouldering in Smichov
49                            Randa Boulder bouldering 
50                            Randa Boulder bouldering 
54    TRADITIONAL kayaking in Utrecht town and Ameli...
55                  Hike (17km) in Mertert/Wasserbillig
56    TRADITIONAL kayaking in Utrecht town and Ameli...
59    Hike in National Park the Loonse en Drunense D...
60    LUXEMBOURG: camping weekend together : Penteco...
61                        Hiking in Germany : Monschau 
Name: title, dtype: object